In [ ]:
from neo4j import GraphDatabase
import pandas as pd
driver=GraphDatabase.driver(uri="bolt://localhost:7687",auth=("neo4j","biomedical_kg"))
session=driver.session()

In [ ]:
# # geo_downregulates = pd.read_csv('geo_downregulates_relationship.tsv', sep='\t')
# # geo_upregulates = pd.read_csv('geo_upregulates_relationship.tsv', sep='\t')
# # string_node_properties = pd.read_csv('string_protein_node.tsv', sep='\t',index_col=[0])
# string_node_relationships = pd.read_csv('STRING/string_interacts-with_relationship.tsv', sep='\t')

Fill null values with "NaN" since Neo4J doesn't accept null values

In [ ]:
# print(string_node_relationships.isna().sum())
# string_node_relationships.fillna('NaN', inplace=True)
# print(string_node_relationships.isna().sum())
# string_node_relationships = string_node_relationships[:10000]
# string_node_relationships.to_csv('STRING/string_interacts-with_relationship_10000.tsv', sep="\t", index=False)
# string_node_relationships

In [ ]:
# Call dbms.listConfig();

STRING Relationship

In [ ]:
q1 = """LOAD CSV WITH HEADERS FROM 'file:///string_interacts-with_relationship.tsv' as row FIELDTERMINATOR '\t'
MERGE (k:protein1 {Name: row.Protein1})
MERGE (j:protein2 {Name: row.Protein2})
MERGE (k) -[:interacts_with {combined_score:row.combined_score, neighborhood:row.neighborhood, fusion:row.fusion, cooccurence:row.cooccurence, coexpression:row.coexpression, database:row.database, textmining:row.textmining}] -> (j)"""

In [ ]:
results=session.run(q1)
data=results.data()
data

Set properties of proteins


In [ ]:
q2 = """LOAD CSV WITH HEADERS FROM 'file:///string_protein_node.tsv' as csvLine FIELDTERMINATOR '\t'
MATCH (k {Name: csvLine.Ensembl_ID})
SET k.Preferred_name = csvLine.Preferred_name, k.Ensembl_HGNC = csvLine.Ensembl_HGNC, k.Ensembl_gene = csvLine.Ensembl_gene, k.Ensembl_UniProt_AC = csvLine.Ensembl_UniProt_AC"""

In [ ]:
results=session.run(q2)
data=results.data()
data

GEO - Downregulation

In [ ]:
q3 = """LOAD CSV WITH HEADERS FROM 'file:///geo_downregulates_relationship.tsv' as row FIELDTERMINATOR '\t'
MERGE (x:disease {Name: row.Disease})
MERGE (y:gene {Name: row.Gene})
MERGE (x) -[:downregulates {metap:row.metap, metafc:row.metafc}] -> (y)
"""

In [ ]:
results=session.run(q3)
data=results.data()
data

GEO - Upregulation

In [ ]:
q4 = """LOAD CSV WITH HEADERS FROM 'file:///geo_upregulates_relationship.tsv' as row FIELDTERMINATOR '\t'
MERGE (x:disease {Name: row.Disease})
MERGE (z:gene {Name: row.Gene})
MERGE (x) -[:upregulates {metap:row.metap, metafc:row.metafc}] -> (z)
"""

In [ ]:
results=session.run(q4)
data=results.data()
data

Forming a heterogeneous dataset of GEO & STRING

In [ ]:
# q5 = """MATCH (a:gene),(b:protein1)
# WHERE a.Name = b.Preferred_name
# CREATE (a)-[r:product]->(b)
# RETURN a,b"""

In [ ]:
# q5_1 = """MATCH (a:gene),(b:protein2)
# WHERE a.Name = b.Preferred_name
# CREATE (a)-[r:product]->(b)
# RETURN a,b"""

In [ ]:
# results=session.run(q5)
# data=results.data()

# results=session.run(q5_1)
# data=results.data()

# data

LINCS

In [ ]:
# down_lincs = pd.read_csv('LINCS/lincs_downregulates_relationship.tsv', sep='\t')
# up_lincs = pd.read_csv('LINCS/lincs_upregulates_relationship.tsv', sep='\t')

In [ ]:
q6 = """LOAD CSV WITH HEADERS FROM 'file:///lincs_downregulates_relationship.tsv' as row FIELDTERMINATOR '\t'
MERGE (x:drug {Name: row.drug})
MERGE (y:gene {Name: row.gene})
MERGE (x) -[:downregulates {zvalue:row.zvalue, pvalue:row.pvalue}] -> (y)
"""

In [ ]:
results=session.run(q6)
data=results.data()
data

In [ ]:
q7 = """LOAD CSV WITH HEADERS FROM 'file:///lincs_upregulates_relationship.tsv' as row FIELDTERMINATOR '\t'
MERGE (x:drug {Name: row.drug})
MERGE (y:gene {Name: row.gene})
MERGE (x) -[:upregulates {zvalue:row.zvalue, pvalue:row.pvalue}] -> (y)
"""

In [ ]:
results=session.run(q7)
data=results.data()
data

DisGeNET

In [ ]:
# dis_rel = pd.read_csv('DisGeNet/disgenet_association_relationship.tsv', sep='\t',index_col=[0])
# dis_disease_node = pd.read_csv('DisGeNet/disgenet_disease_node.tsv', sep='\t',index_col=[0])
# dis_gene_node = pd.read_csv('DisGeNet/disgenet_gene_node.tsv', sep='\t',index_col=[0])

In [ ]:
# dis_gene_node.fillna('NaN', inplace=True)

In [ ]:
# dis_rel.to_csv('DisGeNet/disgenet_association_relationship.tsv', sep="\t", index=False)
# dis_disease_node.to_csv('DisGeNet/disgenet_disease_node.tsv', sep="\t", index=False)
# dis_gene_node.to_csv('DisGeNet/disgenet_gene_node.tsv', sep="\t", index=False)

DisGeNET association

In [ ]:
q8 = """LOAD CSV WITH HEADERS FROM 'file:///disgenet_association_relationship.tsv' as row FIELDTERMINATOR '\t'
MERGE (x:disease {Name: row.disease})
MERGE (y:gene {Name: row.gene})
MERGE (x) -[:associated_with {score:row.score, ei:row.EI, year_initial:row.year_initial, year_final:row.year_final}] -> (y)
"""

In [ ]:
results=session.run(q8)
data=results.data()
data

Set property of disease

In [ ]:
q9 = """LOAD CSV WITH HEADERS FROM 'file:///disgenet_disease_node.tsv' as csvLine FIELDTERMINATOR '\t'
MATCH (k {Name: csvLine.disease})
SET k.diseaseID = csvLine.diseaseID, k.disease_class = csvLine.disease_class, k.disease_class_name = csvLine.disease_class_name, k.disease_semantic_type = csvLine.disease_semantic_type"""

In [ ]:
results=session.run(q9)
data=results.data()
data

Set property of genes

In [ ]:
q10 = """LOAD CSV WITH HEADERS FROM 'file:///disgenet_gene_node.tsv' as csvLine FIELDTERMINATOR '\t'
MATCH (k {Name: csvLine.gene})
SET k.geneID = csvLine.geneID, k.Protein_Class = csvLine.Protein_Class, k.UniProtID = csvLine.UniProtID, k.DPI = csvLine.DPI, k.DSI = csvLine.DSI"""

In [ ]:
results=session.run(q10)
data=results.data()
data

Forming a heterogeneous dataset of LINCS & STRING

In [ ]:
# q11 = """MATCH (a:gene),(b:protein1)
# WHERE a.Name = b.Preferred_name
# CREATE (a)-[r:product]->(b)"""

In [ ]:
# q11_1 = """MATCH (a:gene),(b:protein2)
# WHERE a.Name = b.Preferred_name
# CREATE (a)-[r:product]->(b)"""

In [ ]:
# results=session.run(q11)
# data=results.data()

# results=session.run(q11_1)
# data=results.data()

# data

Forming a heterogeneous dataset of DisGeNET & STRING

In [ ]:
q12 = """MATCH (a:gene),(b:protein)
WHERE a.Name = b.Preferred_name
CREATE (a)-[r:product]->(b)"""

In [ ]:
# q12_1 = """MATCH (a:gene),(b:protein1)
# WHERE a.Name = b.Preferred_name
# CREATE (a)-[r:product]->(b)"""

In [ ]:
results=session.run(q12)
data=results.data()

# results=session.run(q12_1)
# data=results.data()

data

In [ ]:
q13 = """MATCH (n:protein1)
REMOVE n:protein1
SET n:protein"""

q13_1 = """MATCH (n:protein2)
REMOVE n:protein2
SET n:protein"""

results=session.run(q13)
data=results.data()

results=session.run(q13_1)
data=results.data()